<a href="https://colab.research.google.com/github/LostUniavan/MLP-to-image/blob/main/MLPXOR_to_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import numpy as np
import csv
import time

def prepare_csv(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        for i, row in enumerate(reader):
            label = i
            row.append(label)
            writer.writerow(row)

class MLP:
    def __init__(self, input_size, hidden_layers, output_size, learning_rate, error_threshold):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.error_threshold = error_threshold

        self.weights = []
        self.biases = []

        layer_sizes = [input_size] + hidden_layers + [output_size]
        for i in range(len(layer_sizes) - 1):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i+1]) * np.sqrt(2 / layer_sizes[i]))
            self.biases.append(np.zeros((1, layer_sizes[i+1])))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward_propagate(self, inputs):
        activations = [inputs]
        for i in range(len(self.weights)):
            net_inputs = np.dot(activations[i], self.weights[i]) + self.biases[i]
            activations.append(self.sigmoid(net_inputs))
        return activations

    def backward_propagate(self, activations, expected_output):
        deltas = [None] * len(self.weights)
        errors = expected_output - activations[-1]
        deltas[-1] = errors * self.sigmoid_derivative(activations[-1])

        for i in reversed(range(len(deltas) - 1)):
            errors = np.dot(deltas[i + 1], self.weights[i + 1].T)
            deltas[i] = errors * self.sigmoid_derivative(activations[i + 1])

        for i in range(len(self.weights)):
            self.weights[i] += self.learning_rate * np.dot(activations[i].T, deltas[i])
            self.biases[i] += self.learning_rate * np.sum(deltas[i], axis=0)

    def train(self, inputs, expected_outputs, epochs):
        start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        print(f"Treinamento começou em: {start_time}")

        for epoch in range(epochs):
            activations = self.forward_propagate(inputs)
            self.backward_propagate(activations, expected_outputs)

            loss = np.mean((expected_outputs - activations[-1]) ** 2)
            print(f"Epoca {epoch + 1}/{epochs}, Erro: {loss}")

            if loss < self.error_threshold:
                break

        end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        print(f"Treinamento finalizou em: {end_time}")

    def predict(self, inputs):
        activations = self.forward_propagate(inputs)
        return activations[-1]

def apply_visual_noise(data, noise_level):
    noisy_data = data.copy()
    num_elements = noisy_data.size

    ones_indices = np.where(noisy_data == 1)[0]
    zeros_indices = np.where(noisy_data == 0)[0]

    num_noisy_ones = int(len(ones_indices) * noise_level)
    num_noisy_zeros = int(len(zeros_indices) * noise_level)

    noisy_ones_indices = np.random.choice(ones_indices, num_noisy_ones, replace=False)
    noisy_zeros_indices = np.random.choice(zeros_indices, num_noisy_zeros, replace=False)

    for index in noisy_ones_indices:
        noisy_data.flat[index] = 0
    for index in noisy_zeros_indices:
        noisy_data.flat[index] = 1

    return noisy_data

def evaluate_mlp_with_visual_noise(mlp, test_data, expected_outputs, noise_levels):
    for noise_level in noise_levels:
        noisy_inputs = np.array([apply_visual_noise(x, noise_level) for x in test_data])
        correct, incorrect = evaluate_mlp(mlp, noisy_inputs, expected_outputs)
        print(f"Nível de ruído: {noise_level*100}%, Correto: {correct}, Incorreto: {incorrect}")

def evaluate_mlp(mlp, test_data, expected_outputs):
    correct_predictions = 0
    total_predictions = len(test_data)

    for inputs, expected_output in zip(test_data, expected_outputs):
        prediction = mlp.predict(inputs)
        if np.argmax(prediction) == np.argmax(expected_output):
            correct_predictions += 1

    return correct_predictions, total_predictions - correct_predictions

input_file = 'data_input.csv'
output_file = 'data_input_preparado.csv'
prepare_csv(input_file, output_file)

data = np.loadtxt(output_file, delimiter=',')
inputs = data[:, :-1]
num_classes = len(np.unique(data[:, -1]))
expected_outputs = data[:, -1]

mlp = MLP(input_size=64, hidden_layers=[64, 32], output_size=num_classes, learning_rate=0.01, error_threshold=1e-2)
mlp.train(inputs, expected_outputs, epochs=5000)

correct, incorrect = evaluate_mlp(mlp, inputs, expected_outputs)
print(f"Teste com os dados de treinamento vs resultado esperado, Correto: {correct}, Incorreto: {incorrect}")

noise_levels = [0.02, 0.05, 0.1, 0.3]
evaluate_mlp_with_visual_noise(mlp, inputs, expected_outputs, noise_levels)


Treinamento começou em: 2024-07-05 01:14:36
Epoca 1/5000, Erro: 14490.66329339765
Epoca 2/5000, Erro: 14387.16215481051
Epoca 3/5000, Erro: 14387.161961623557
Epoca 4/5000, Erro: 14387.16195094091
Epoca 5/5000, Erro: 14387.161944675718
Epoca 6/5000, Erro: 14387.161940398084
Epoca 7/5000, Erro: 14387.161937221876
Epoca 8/5000, Erro: 14387.161934734162
Epoca 9/5000, Erro: 14387.161932712374
Epoca 10/5000, Erro: 14387.161931024173
Epoca 11/5000, Erro: 14387.161929585109
Epoca 12/5000, Erro: 14387.161928338282
Epoca 13/5000, Erro: 14387.16192724372
Epoca 14/5000, Erro: 14387.16192627235
Epoca 15/5000, Erro: 14387.161925402444
Epoca 16/5000, Erro: 14387.16192461734
Epoca 17/5000, Erro: 14387.161923904041
Epoca 18/5000, Erro: 14387.161923252204
Epoca 19/5000, Erro: 14387.1619226535
Epoca 20/5000, Erro: 14387.161922101093
Epoca 21/5000, Erro: 14387.161921589355
Epoca 22/5000, Erro: 14387.161921113568
Epoca 23/5000, Erro: 14387.161920669754
Epoca 24/5000, Erro: 14387.161920254533
Epoca 25/5000

<ipython-input-94-41bce7a31118>:32: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))



Epoca 41/5000, Erro: 14387.161915770883
Epoca 42/5000, Erro: 14387.161915598756
Epoca 43/5000, Erro: 14387.161915432944
Epoca 44/5000, Erro: 14387.161915273082
Epoca 45/5000, Erro: 14387.161915118837
Epoca 46/5000, Erro: 14387.161914969902
Epoca 47/5000, Erro: 14387.16191482598
Epoca 48/5000, Erro: 14387.161914686816
Epoca 49/5000, Erro: 14387.161914552156
Epoca 50/5000, Erro: 14387.161914421775
Epoca 51/5000, Erro: 14387.161914295455
Epoca 52/5000, Erro: 14387.161914172997
Epoca 53/5000, Erro: 14387.161914054223
Epoca 54/5000, Erro: 14387.161913938951
Epoca 55/5000, Erro: 14387.161913827022
Epoca 56/5000, Erro: 14387.161913718282
Epoca 57/5000, Erro: 14387.161913612586
Epoca 58/5000, Erro: 14387.1619135098
Epoca 59/5000, Erro: 14387.161913409805
Epoca 60/5000, Erro: 14387.161913312477
Epoca 61/5000, Erro: 14387.161913217698
Epoca 62/5000, Erro: 14387.161913125368
Epoca 63/5000, Erro: 14387.161913035392
Epoca 64/5000, Erro: 14387.161912947664
Epoca 65/5000, Erro: 14387.161912862106
Ep